In [1]:
import tensorflow as tf
import cifar10_input
batch_size = 64

In [2]:
images_trains, labels_trains = cifar10_input.distorted_inputs(data_dir="../cifar_data/cifar-10-batches-bin/",
                                                              batch_size=batch_size)

images_test, labels_test = cifar10_input.inputs(eval_data=True,data_dir="../cifar_data/cifar-10-batches-bin/",
                                               batch_size=batch_size)

Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(string_tensor).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(input_tensor).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensors(tensor).repeat(num_epochs)`.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.FixedLengthRecordDataset`.
Filling queue with 20000 CIF

In [3]:
def variable_weight(shape, stddev=5e-2):
    init = tf.truncated_normal_initializer(stddev=stddev)
    return tf.get_variable(initializer=init, shape=shape, name='weight')

def variable_bias(shape):
    init = tf.constant_initializer(0.1)
    return tf.get_variable(initializer=init, shape=shape, name='bias')

def conv(x, ksize, out_depth, strides, padding='SAME', act=tf.nn.relu, scope='conv_layer', reuse=None):
    in_depth = x.get_shape().as_list()[-1]
    with tf.variable_scope(scope, reuse=reuse):
        shape = ksize+[in_depth, out_depth]
        with tf.variable_scope('kernel'):
            kernel = variable_weight(shape=shape)
        strides = [1, strides[0], strides[1],1]
        conv = tf.nn.conv2d(x, kernel, strides, padding, name='conv')
        with tf.variable_scope('bias'):
            bias = variable_bias([out_depth])
        preact = tf.nn.bias_add(conv,bias)
        out = act(preact)
        return out

def max_pool(x, ksize, strides, padding='SAME', name='pool_layer'):
    return tf.nn.max_pool(x,[1, ksize[0], ksize[1],1],[1,strides[0], strides[1],1], padding, name=name)

def fc(x, out_depth, act=tf.nn.relu, scope='fully_connect', reuse=None):
    in_depth = x.get_shape().as_list()[-1]
    with tf.variable_scope(scope, reuse=reuse):
        with tf.variable_scope('weight'):
            weitht = variable_weight([in_depth, out_depth])
        with tf.variable_scope('bias'):
            bias= variable_bias([out_depth])
        fc = tf.nn.bias_add(tf.matmul(x,weitht),bias, name='fc')
        out = act(fc)
        
        return out


In [4]:
#真正意义上的深层网络结构， imageNet2014亚军
def vgg_block(inputs, num_convs,out_depth, scope="vgg_block", reuse=None ):
    #一个block有num_convs个卷积层和一个最大池化层组成
    input_depth = inputs.get_shape().as_list()[-1]
    with tf.variable_scope(scope, reuse=reuse) as sc:
        net = inputs
        for i in range(num_convs):
            net = conv(net, ksize=[3,3], out_depth=out_depth, strides=[1,1], padding='SAME', 
                       scope='conv%d' % i,reuse=reuse)
        net = max_pool(net, ksize=[2,2], strides=[2,2], name='pool')
        
        return net

In [5]:
def vgg_stack(inputs, num_convs, out_depths, scope = 'vgg_stack', reuse=None):
    #输入， 每个block卷积层的个数； 每个block卷积核的个数
    with tf.variable_scope(scope,reuse=reuse) as sc:
        net = inputs
        for i , (n,d) in enumerate(zip(num_convs, out_depths)):
            net = vgg_block(net, n, d, scope='block%d' % i)
        return net

In [6]:
#加入全链接搭建vgg
def vgg(inputs, num_convs,out_depths, num_outputs, scope='vgg', reuse=None):
    #经过stack后连接两个全连接层
    net = inputs
    with tf.variable_scope(scope, reuse=reuse) as sc:
        net = vgg_stack(net, num_convs=num_convs, out_depths=out_depths)
        with tf.variable_scope('classification'):
            net = tf.reshape(net,(batch_size,-1))
            net = fc(net, 100, scope='fc1')
            net = fc(net, num_outputs, act=tf.identity, scope='classification')
            
        return net

In [7]:
image_holder = tf.placeholder(shape=[batch_size,24,24,3], dtype=tf.float32)
label_holder = tf.placeholder(shape=[batch_size], dtype=tf.int32)

In [8]:
train_out = vgg(image_holder, (1,1,2,2,2), (64,128,256,512,512), 10)

In [9]:
with tf.variable_scope('loss'):
    train_loss = tf.reduce_mean(tf.losses.sparse_softmax_cross_entropy(logits=train_out, 
                                                                       labels=label_holder, scope='train'))

In [10]:
with tf.variable_scope('accuracy'):
    with tf.name_scope('tran'):
        train_acc = tf.reduce_mean(tf.cast(tf.equal(tf.argmax(train_out, axis=-1, output_type=tf.int32),
                                                    label_holder),tf.float32))

In [11]:
lr = 1e-3
#train_op = tf.train.MomentumOptimizer(learning_rate=lr, momentum=0.9).minimize(train_loss)
train_op = tf.train.AdamOptimizer(learning_rate=lr, beta1=0.9, beta2=0.999, epsilon=1e-08).minimize(train_loss)

In [12]:
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())

In [13]:
tf.train.start_queue_runners()

Instructions for updating:
To construct input pipelines, use the `tf.data` module.


[<Thread(QueueRunnerThread-input_producer-input_producer/input_producer_EnqueueMany, started daemon 139981188208384)>,
 <Thread(QueueRunnerThread-shuffle_batch/random_shuffle_queue-shuffle_batch/random_shuffle_queue_enqueue, started daemon 139980551530240)>,
 <Thread(QueueRunnerThread-shuffle_batch/random_shuffle_queue-shuffle_batch/random_shuffle_queue_enqueue, started daemon 139980543137536)>,
 <Thread(QueueRunnerThread-shuffle_batch/random_shuffle_queue-shuffle_batch/random_shuffle_queue_enqueue, started daemon 139980534744832)>,
 <Thread(QueueRunnerThread-shuffle_batch/random_shuffle_queue-shuffle_batch/random_shuffle_queue_enqueue, started daemon 139980526352128)>,
 <Thread(QueueRunnerThread-shuffle_batch/random_shuffle_queue-shuffle_batch/random_shuffle_queue_enqueue, started daemon 139980517959424)>,
 <Thread(QueueRunnerThread-shuffle_batch/random_shuffle_queue-shuffle_batch/random_shuffle_queue_enqueue, started daemon 139980164822784)>,
 <Thread(QueueRunnerThread-shuffle_batch/

In [14]:
for i in range(1000):
    image_bath, label_batch = sess.run([images_trains, labels_trains])
    _, loss1, acc1 = sess.run([train_op,train_loss,train_acc], feed_dict={image_holder: image_bath, label_holder: label_batch})
    if i % 10 ==0:
        print("Step%d  train loss %.6f train acc: %.6f" % (i+1, loss1, acc1))

Step1  train loss 3.398097 train acc: 0.140625
Step11  train loss 2.321986 train acc: 0.093750
Step21  train loss 2.289397 train acc: 0.109375
Step31  train loss 2.440855 train acc: 0.062500
Step41  train loss 2.200521 train acc: 0.281250
Step51  train loss 2.125059 train acc: 0.125000
Step61  train loss 2.129168 train acc: 0.156250
Step71  train loss 2.086580 train acc: 0.171875
Step81  train loss 2.112401 train acc: 0.187500
Step91  train loss 2.278561 train acc: 0.125000
Step101  train loss 2.164048 train acc: 0.125000
Step111  train loss 2.117632 train acc: 0.187500
Step121  train loss 1.972025 train acc: 0.218750
Step131  train loss 2.000384 train acc: 0.234375
Step141  train loss 1.958821 train acc: 0.281250
Step151  train loss 1.947047 train acc: 0.234375
Step161  train loss 1.846113 train acc: 0.296875
Step171  train loss 2.079134 train acc: 0.187500
Step181  train loss 1.882857 train acc: 0.265625
Step191  train loss 2.044087 train acc: 0.234375
Step201  train loss 1.895617 tr